# Teste com presidio sem adicionar os padrões do projeto

In [1]:
import os

In [145]:
path = '../dados-teste/'

dirs = os.listdir(path)

print(dirs)

['resultado-concursos', 'cnj', 'diario-oficial-estado-rj']


In [146]:
path_cnj = os.path.join(path, dirs[1])
path_ioerj = os.path.join(path, dirs[2])
path_concurso = os.path.join(path, dirs[0])

In [147]:
def get_only_txt(files):
    l = []

    for file in files:
        if file[-3:] == 'txt':
            l.append( file )
    
    return l


files_cnj = get_only_txt( os.listdir(path_cnj) )
files_ioerj = get_only_txt( os.listdir(path_ioerj) )
files_concurso = get_only_txt( os.listdir(path_concurso))

In [148]:
scores = [0.2, 0.5, 0.8, 1.0]

In [2]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider

In [4]:
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

configuration = {
    "nlp_engine_name": "spacy",
    "models": [
        # {"lang_code": "en", "model_name": "en_core_web_lg"},
        {"lang_code": "pt", "model_name": "pt_core_news_lg"},
    ],
}

provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine = provider.create_engine()

analyzer = AnalyzerEngine(
    nlp_engine=nlp_engine, supported_languages=["pt"]
)

In [5]:
def analise_presidio_block_of_files_by_source(base_path,files):
    print(base_path)
    for file in files:
        for score in scores:
            print(80*'-')
            print(f'score: {score} - {file}', end='')
            with open(file=os.path.join(base_path, file), mode='r') as f:
                content = f.read()

                # result = analyzer.analyze(text=content, language="en",score_threshold=score)
                result = []
                result += analyzer.analyze(text=content, language="pt",score_threshold=score)

                print(f' - PII detectados: {len(result)}')
                
        
        print(140*'*')

In [27]:
a = '1415926535897905852807000010546264338327950288419716939937510582097494459230781640628620899862803482534211706798214808651328230664709384460955058223172535940812848111745028410270193852110555964462294895493038196.'
a = '''042.815.546-45 08/06/1952
2 ADAO DA SILVA
262.087.846-20 24/03/1951
3 ADAO LUIZ FERREIRA
156.077.258-12 21/02/1936
4 ADELINA DOS SANTOS VALHAS
130.499.326-43 20/03/1994'''

# analyzer.analyze(text=a, language="pt",score_threshold=0.8)
b = check_prompt(text=a, score_threshold=0.1)

In [28]:
b

([{'type': 'BR_CPF', 'score': 1.0},
  {'type': 'BR_CPF', 'score': 1.0},
  {'type': 'BR_CPF', 'score': 1.0},
  {'type': 'BR_CPF', 'score': 1.0},
  {'type': 'DATE_TIME', 'score': 0.6},
  {'type': 'DATE_TIME', 'score': 0.6},
  {'type': 'DATE_TIME', 'score': 0.6},
  {'type': 'DATE_TIME', 'score': 0.6},
  {'type': 'BR_RG', 'score': 0.4},
  {'type': 'BR_RG', 'score': 0.4},
  {'type': 'BR_RG', 'score': 0.4},
  {'type': 'BR_RG', 'score': 0.4}],
 '<BR_CPF> <DATE_TIME>\n2 ADAO DA SILVA\n<BR_CPF> <DATE_TIME>\n3 ADAO LUIZ FERREIRA\n<BR_CPF> <DATE_TIME>\n4 ADELINA DOS SANTOS VALHAS\n<BR_CPF> <DATE_TIME>')

In [152]:
print('Presidio')

analise_presidio_block_of_files_by_source(path_cnj, files_cnj)
analise_presidio_block_of_files_by_source(path_ioerj, files_ioerj)
analise_presidio_block_of_files_by_source(path_concurso, files_concurso)

Presidio
../dados-teste/cnj
--------------------------------------------------------------------------------
score: 0.2 - lista1.txt - PII detectados: 471
--------------------------------------------------------------------------------
score: 0.5 - lista1.txt - PII detectados: 471
--------------------------------------------------------------------------------
score: 0.8 - lista1.txt - PII detectados: 471
--------------------------------------------------------------------------------
score: 1.0 - lista1.txt - PII detectados: 0
********************************************************************************************************************************************
../dados-teste/diario-oficial-estado-rj
--------------------------------------------------------------------------------
score: 0.2 - IOERJ-01-abril.txt - PII detectados: 318
--------------------------------------------------------------------------------
score: 0.5 - IOERJ-01-abril.txt - PII detectados: 295
---------------

# Teste com os NER brasileiros add

In [1]:
from presidio_checker import check_prompt

In [154]:
def analise_gaiseco_block_of_files_by_source(base_path, files):
    print(base_path)
    for file in files:
        for score in scores:
            print(80*'-')
            print(f'score: {score} - {file}', end='')
            with open(file=os.path.join(base_path, file), mode='r') as f:
                content = f.read()

                result = check_prompt(text=content, score_threshold=score)

                print(f' - PII detectados: {len(result[0])}', end='')
                print(f' - BR PII detectados: {len( str( result[0] ).split("BR") ) -1}')
                
        
    print(140*'*')

In [155]:
print('GAISeCo')

analise_gaiseco_block_of_files_by_source(path_cnj, files_cnj)
analise_gaiseco_block_of_files_by_source(path_ioerj, files_ioerj)
analise_gaiseco_block_of_files_by_source(path_concurso, files_concurso)

GAISeCo
../dados-teste/cnj
--------------------------------------------------------------------------------
score: 0.2 - lista1.txt - PII detectados: 471 - BR PII detectados: 0
--------------------------------------------------------------------------------
score: 0.5 - lista1.txt - PII detectados: 471 - BR PII detectados: 0
--------------------------------------------------------------------------------
score: 0.8 - lista1.txt - PII detectados: 471 - BR PII detectados: 0
--------------------------------------------------------------------------------
score: 1.0 - lista1.txt - PII detectados: 0 - BR PII detectados: 0
********************************************************************************************************************************************
../dados-teste/diario-oficial-estado-rj
--------------------------------------------------------------------------------
score: 0.2 - IOERJ-01-abril.txt - PII detectados: 331 - BR PII detectados: 13
-----------------------------------